In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NigelPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Prepare Stopwords

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
# Stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(['from', 'subject', 're', 'edu', 'use'])

In [5]:
# Extend stopwords with WR's .txt files
import glob
import re

files = glob.glob('./stopwords/*.txt')
for name in files:
    with open(name, 'r') as f:
        text = f.read()
        words = re.split(r'[;,\s\n]\s*', text)
        stop_words.update(words)

In [6]:
stop_words = list(stop_words)

### Prepare Data

In [7]:
# Import dataset
df = pd.read_json('newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [8]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


### Tokenize words and clean up text

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


### Create Bigram and Trigram Models

In [10]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


### Remove Stopwords, make Bigrams and Lemmatize

In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['bumper', 'body', 'neighborhood']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


### Build LDA Model

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
# Print the top-10 keywords in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.134*"power" + 0.133*"law" + 0.045*"crime" + 0.038*"right" + '
  '0.035*"license" + 0.033*"criminal" + 0.030*"view" + 0.027*"hole" + '
  '0.025*"factor" + 0.022*"property"'),
 (1,
  '0.053*"function" + 0.053*"field" + 0.043*"code" + 0.041*"author" + '
  '0.033*"copy" + 0.028*"paper" + 0.026*"digital" + 0.025*"analysis" + '
  '0.025*"address" + 0.023*"sample"'),
 (2,
  '0.180*"card" + 0.139*"driver" + 0.057*"input" + 0.050*"appreciate" + '
  '0.040*"manual" + 0.035*"material" + 0.035*"mode" + 0.033*"printer" + '
  '0.027*"assumption" + 0.027*"output"'),
 (3,
  '0.209*"government" + 0.084*"israeli" + 0.070*"security" + 0.052*"citizen" + '
  '0.036*"rational" + 0.030*"pass" + 0.028*"arab" + 0.028*"disclaimer" + '
  '0.026*"voice" + 0.024*"law_enforcement"'),
 (4,
  '0.056*"mail" + 0.042*"distribution" + 0.037*"computer" + 0.030*"datum" + '
  '0.028*"software" + 0.027*"technology" + 0.025*"current" + 0.024*"machine" + '
  '0.021*"disk" + 0.019*"error"'),
 (5,
  '0.198*"system" + 0

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -17.690016959867837

Coherence Score:  0.4080619864397727


In [22]:
lda_model.save('lda_model')

### Visualise Topic Keywords

In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

C:\Users\NigelPC\Miniconda3\envs\nlp-ds\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [18]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.309460 -0.204505       1        1  18.092201
18    -0.272278  0.369891       2        1  17.569237
4     -0.213549 -0.112406       3        1  11.403429
5     -0.067347  0.044302       4        1   6.355700
13    -0.107107 -0.148558       5        1   6.354414
12    -0.046149  0.031135       6        1   5.037771
19    -0.036012 -0.008450       7        1   4.907513
14     0.036443  0.003042       8        1   4.347093
17     0.002668 -0.010296       9        1   3.920696
1      0.045065  0.008796      10        1   3.391818
0      0.030883  0.018697      11        1   3.194767
11     0.057641  0.007927      12        1   2.971924
15     0.068231 -0.001563      13        1   2.308695
2      0.076706  0.007051      14        1   2.155948
8      0.085307  0.006075      15        1   2.133689
3      0.078483 -0.012479      16        1   2.025845
9      0.130523  0.001437      17        1   1.306896
6      0.132932  0.001219      18        1   1.167998
7      0.147008 -0.000050      19        1   0.959085
16     0.160013 -0.001263      20        1   0.395281, topic_info=     Category         Freq        Term        Total  loglift  logprob
2685  Default  6401.000000          ax  6401.000000  30.0000  30.0000
71    Default  6030.000000      system  6030.000000  29.0000  29.0000
101   Default  2267.000000  government  2267.000000  28.0000  28.0000
463   Default  2202.000000        will  2202.000000  27.0000  27.0000
8     Default  1855.000000        card  1855.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
2832  Topic20     0.036297    lebanese     1.641603   1.7216 -10.8610
6365  Topic20     0.036296  consistent     1.641528   1.7217 -10.8611
5895  Topic20     0.036296   terrorist     1.641592   1.7216 -10.8611
2472  Topic20     0.036296      chance     1.641529   1.7216 -10.8611
6317  Topic20     0.036295    shipping     1.641415   1.7217 -10.8611

[672 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
464      14  0.736795           abs
465      14  0.956623  acceleration
710       6  0.996300        accept
24        8  0.999091        access
379      17  0.954695     accessory
...     ...       ...           ...
377       1  0.996906          wing
663       7  0.990937   workstation
63        2  0.998013         would
63        8  0.001699         would
1098     20  0.956769         young

[639 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 19, 5, 6, 14, 13, 20, 15, 18, 2, 1, 12, 16, 3, 9, 4, 10, 7, 8, 17])

In [19]:
pyLDAvis.save_html(vis, 'lda.html')